In [28]:
import sqlite3
import json
from datetime import datetime, timedelta

In [29]:
filePath = 'data.json'
dbName = 'bids.db'

In [30]:

def get_json(filePath):
    with open(filePath) as f:
        jsonFile = json.load(f)

    return jsonFile
    

jsonFile = get_json(filePath)


In [31]:
#Creates and connects to database 
#Just connects if it allready exists
#Also get the cursor for the connection

def create_connectt_db(dbName):
    conn = None
    try:
        conn = sqlite3.connect(dbName)
    except sqlite3.Error as err:
        print(err)
    finally:
        if conn:
            return conn


conn = create_connectt_db(dbName)
cursor = conn.cursor()

In [32]:
# Deletes all tables from bids.db if they exist

def drop_database(db_name):
    # Connect to the SQLite database
    conn = sqlite3.connect(db_name)
    cursor = conn.cursor()

    # List of tables to drop
    tables = [
        'series',
        'positions',
        'main'
    ]

    # Drop each table if it exists
    for table in tables:
        cursor.execute(f'DROP TABLE IF EXISTS {table}')

    # Commit the changes and close the connection
    conn.commit()
    conn.close()
    print('Database tables dropped successfully.')

# Example usage
drop_database(dbName)

Database tables dropped successfully.


In [33]:
#Create the tables

# Create main table
cursor.execute('''
CREATE TABLE IF NOT EXISTS main (
    externalId TEXT PRIMARY KEY,
    day TEXT,
    dateOfLastChange TEXT,
    market TEXT,
    status TEXT,
    country TEXT,
    updateTime TEXT,
    fromStatus TEXT,
    toStatus TEXT
)
''')

# Create series table
cursor.execute('''
CREATE TABLE IF NOT EXISTS series (
    seriesExternalId TEXT PRIMARY KEY,
    externalId TEXT,
    customerId TEXT,
    status TEXT,
    direction TEXT,
    currency TEXT,
    priceArea TEXT,
    assetId TEXT,
    price REAL,
    startInterval TEXT,
    endInterval TEXT,
    resolution TEXT,
    FOREIGN KEY (externalId) REFERENCES main (externalId)
)
''')

# Create positions table
cursor.execute('''
CREATE TABLE IF NOT EXISTS positions (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    seriesExternalId TEXT,
    quantity INTEGER,
    dateTime TIMESTAMP,
    FOREIGN KEY (seriesExternalId) REFERENCES series (seriesExternalId)
)
''')



In [34]:
# Helper functions from turning a string to datetime format, and the other way around

def str_to_date_time(dateTimeString):
    return datetime.strptime(dateTimeString, "%Y-%m-%dT%H:%M:%S")

def date_time_to_str(dateTime):
    return dateTime.strftime("%Y-%m-%dT%H:%M:%S")

In [35]:

# Insert data into main table
cursor.execute('''
INSERT OR IGNORE INTO main (externalId, day, dateOfLastChange, market, status, country, updateTime, fromStatus, toStatus)
VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?)
''', (
    jsonFile['externalId'],
    jsonFile['day'],
    jsonFile['dateOfLastChange'],
    jsonFile['market'],
    jsonFile['status'],
    jsonFile['country'],
    jsonFile['updateHistory'][0]['updateTime'],
    jsonFile['updateHistory'][0]['fromStatus'],
    jsonFile['updateHistory'][0]['toStatus']
))

# Insert data into series and positions tables
for series in jsonFile['series']:
    cursor.execute('''
    INSERT INTO series (externalId, seriesExternalId, customerId, status, direction, currency, priceArea, assetId, price, startInterval, endInterval, resolution)
    VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
    ''', (
        jsonFile['externalId'],
        series['externalId'],
        series['customerId'],
        series['status'],
        series['direction'],
        series['currency'],
        series['priceArea'],
        series['assetId'],
        series['price'],
        series['startInterval'],
        series['endInterval'],
        series['resolution']
    ))
    
    # Get the start time as a string
    dateTime_start_str = series['startInterval']

    # Convert the start time string to datetime
    dateTimeCurrent = str_to_date_time(dateTime_start_str)
    
    for position in series['positions']:
        cursor.execute('''
        INSERT INTO positions (seriesExternalId, quantity, dateTime)
        VALUES (?, ?, ?)
        ''', (
            series['externalId'],
            position['quantity'],
            dateTimeCurrent
        ))
        dateTimeCurrent += timedelta(hours=1)
        



C:\Users\Trym\AppData\Local\Temp\ipykernel_35940\2840564174.py:44: DeprecationWarning: The default datetime adapter is deprecated as of Python 3.12; see the sqlite3 documentation for suggested replacement recipes
  cursor.execute('''


In [36]:
# Commit changes and close the connection
conn.commit()
conn.close()